### Tensorboard
Tensorboard is the most popular tool for tracking the training of neural networks in either Tensorflow or PyTorch. In fact, it can be used as a visualization tool for many types of data-logs.
In this notebook we see how to work with Tensorboard while using PyTorch, and we will review most of the types of data that can be logged for visualization in Tensorboard.

In this notebook we will demonstrate how to use _Tensorboard_ for tracking the training. We pick-up from where we left the linear regression problem with the `nn.Module`. 

#### Optional Resources

* [Tensorboard Homepage](https://www.tensorflow.org/tensorboard)
* [Tensorboard.dev](https://tensorboard.dev/) - a service by Google, which allows to freely (and _publicly_) share Tensorboard dashboards.

PyTorch tutorials about Tensorboard integration:
* https://pytorch.org/docs/stable/tensorboard.html
* https://pytorch.org/tutorials/intermediate/tensorboard_tutorial.html
* https://pytorch.org/tutorials/recipes/recipes/tensorboard_with_pytorch.html

In [1]:
import torch
import numpy as np
from torch.utils.tensorboard import SummaryWriter

### Setting up the Pytorch Tensorboard writer
* This requires Tensorboard to be installed (via `pip install`). In this repository we install it as part of the `requirements.txt` files.
* Tensorflow supports tracking and reporting of: scalars, images, histograms, graphs, and embedding visualizations.
* The `SummaryWriter` class is the main entry point to log data for consumption and visualization by TensorBoard.
* `SummaryWriter` is writing the data every time it is called. Commonly, this takes place every given number of training iterations. This way, Tensorflow can show data as it is being generated during the training. By the way, it updates the file contents asynchronously. This allows a training program to call methods to add data to the file directly from the training loop, without slowing down training.
* For more information see: [Tensorboard with PyTorch](https://pytorch.org/docs/stable/tensorboard.html)

In this notebook we are running dummy experiments.
The first thing we notice is that Tensorboard can consume information that is saved to disk in the right format, Tensorboard will be able to present it.

In [2]:
writer = SummaryWriter('runs/dummy_experiment')  # default `log_dir` is "runs" - we'll be more specific here

Below we review the many data types that can be logged for visualization with Tensorboard. Make sure to open Tensorboard, and view the generated output following the execution of each cell. This can be done via a Web Browser or directly within VSCode if that's your IDE. 

##### `add_scalar`
Note: notice that the `writer` writes is called on every iteration. When implemented this way as part of a training loop, the data is continuously written to disk, and is presentable in Tensorboard in near real-time. Caveat - note that the Tensorboard might need to be refreshed, or the data might not be immediatly written. To force a write (which is usually not needed) the `flush` method can be used. 

In [3]:
for n_iter in range(100):
    writer.add_scalar('Loss/train', np.random.random(), n_iter)
    writer.add_scalar('Loss/test', np.random.random(), n_iter)
    writer.add_scalar('Accuracy/train', np.random.random(), n_iter)
    writer.add_scalar('Accuracy/test', np.random.random(), n_iter)

In [4]:
x = range(100)
for i in x:
    writer.add_scalar('y=2x', i * 2, i)

##### `add_scalars`
This call adds three values to the same scalar plot with the tag 'run_14h' in TensorBoard's scalar section.

In [5]:
r = 5
for i in range(100):
    writer.add_scalars('run_14h', {'xsinx':i*np.sin(i/r),
                                    'xcosx':i*np.cos(i/r),
                                    'tanx': np.tan(i/r)}, i)

##### `add_histogram`

In [6]:
writer = SummaryWriter()
for i in range(10):
    x = np.random.random(1000)
    writer.add_histogram('distribution centers', x + i, i)

##### `add_image`
See [Tensorboard with PyTorch](https://pytorch.org/docs/stable/tensorboard.html) for input convention

In [7]:
img = np.zeros((3, 100, 100))
img[0] = np.arange(0, 10000).reshape(100, 100) / 10000
img[1] = 1 - np.arange(0, 10000).reshape(100, 100) / 10000

writer.add_image('my_image', img, 0)

##### `add_text`

In [8]:
writer.add_text(tag='lstm', text_string='This is an lstm', global_step=0)
writer.add_text(tag='rnn', text_string='This is an rnn', global_step=10)

##### `add_embeddings`

In [9]:
import keyword

meta = []
while len(meta)<100:
    meta = meta+keyword.kwlist # get some strings
meta = meta[:100]

for i, v in enumerate(meta):
    meta[i] = v+str(i)

label_img = torch.rand(100, 3, 10, 32)
for i in range(100):
    label_img[i]*=i/100.0

writer.add_embedding(torch.randn(100, 5), metadata=meta, label_img=label_img)
writer.add_embedding(torch.randn(100, 5), label_img=label_img)
writer.add_embedding(torch.randn(100, 5), metadata=meta)

##### `add_pr_curve`

In [10]:
labels = np.random.randint(2, size=100)  # binary label
predictions = np.random.rand(100)
writer = SummaryWriter()
writer.add_pr_curve('pr_curve', labels, predictions, 0)

##### `add_hparams`

In [11]:
with SummaryWriter() as w:
    for i in range(5):
        w.add_hparams({'lr': 0.1*i, 'bsize': i},
                      {'hparam/accuracy': 10*i, 'hparam/loss': 10*i})

##### `add_graph`

`add_graph` accepts a [torch.nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module) as its `Model` argument. 
Let's define one so that we are able to watch the resulting computational graph.

In [12]:
from torch import nn

In [13]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

In [14]:
writer.add_graph(model=model, input_to_model=torch.rand(1, 28*28))

#### Additional Types of Data Loggers

Notably, using PyTorch `SummaryWriter` it is possible to log to Tensorboard video and audio clips as well (by using `add_video` and `add_audio`).
For a complete and up-to-date list of all data types that can be logged, see 

In [15]:
# it's always a good idea to close the writer when done using it
writer.close()